In [1]:
# Install the necessary packages
%pip install -qU langchain-community langchain-openai pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00


In [2]:
# Import libraries
import yaml
from langchain_community.agent_toolkits import JsonToolkit, create_json_agent
from langchain_community.tools.json.tool import JsonSpec
from langchain_openai import OpenAI
import json

In [4]:
# Import and setup keys
import os
import getpass
from google.colab import userdata
os.environ["OPENAI_API_KEY"]=userdata.get('OPENAI_API_KEY')

In [5]:
# Fake JSON data for different categories
brand_data = {
    "shoes": ["Nike", "Adidas", "Puma", "Reebok", "Under Armour", "Converse", "New Balance", "Vans", "Asics", "Fila"],
    "watches": ["Rolex", "Omega", "Cartier", "Tag Heuer", "Seiko", "Citizen", "Casio", "Tissot", "Breitling", "Fossil"],
    "football": ["Nike", "Adidas", "Puma", "Umbro", "New Balance", "Mitre", "Under Armour", "Select", "Wilson", "Diadora"],
    "locks": ["Yale", "Schlage", "Kwikset", "August", "Master Lock", "Medeco", "Mul-T-Lock", "Baldwin", "Weiser", "Defiant"],
}

In [6]:
# Define the JSON spec from the fake database
json_spec = JsonSpec(dict_=brand_data, max_value_length=4000)

# Initialize the JSON Toolkit
json_toolkit = JsonToolkit(spec=json_spec)

In [7]:
# Create a JSON Agent using OpenAI's model
llm = OpenAI(temperature=0)  # Use deterministic outputs

json_agent_executor = create_json_agent(
    llm=llm,
    toolkit=json_toolkit,
    verbose=True
)

In [8]:
# Example 1: Asking about watches
response = json_agent_executor.run(
    "Give me the top 10 brands for watches in key-value JSON format where keys are ranks and values are brand names."
)

print(response)

<ipython-input-8-da49d8e7a6ae>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = json_agent_executor.run(




> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['shoes', 'watches', 'football', 'locks']
Thought: I should look at the keys that exist in data["watches"] to see what I have access to
Action: json_spec_list_keys
Action Input: data["watches"]
Observation: ValueError('Value at path `data["watches"]` is not a dict, get the value directly.')
Thought: I should use json_spec_get_value to see what the value is at data["watches"]
Action: json_spec_get_value
Action Input: data["watches"]
Observation: ['Rolex', 'Omega', 'Cartier', 'Tag Heuer', 'Seiko', 'Citizen', 'Casio', 'Tissot', 'Breitling', 'Fossil']
Thought: I now know the final answer
Final Answer: {1: 'Rolex', 2: 'Omega', 3: 'Cartier', 4: 'Tag Heuer', 5: 'Seiko', 6: 'Citizen', 7: 'Casio', 8: 'Tissot', 9: 'Breitling', 10: 'Fossil'}

> Finished chain.
{1: 'Rolex', 2: 'Omega', 3: 'Cartier', 4: 'Tag Heuer', 5: 'Seiko', 6: 'Citizen', 7: 'Casio', 8: 'Tissot', 9: 'Breitling', 10: 'Fossil'}


In [11]:
products={
  "shoes": {"passage": "Nike, Adidas, Puma, Reebok, Under Armour, Converse, New Balance, Vans, Asics, Fila"},
  "watches": {"passage": "Rolex, Omega, Cartier, Tag Heuer, Seiko, Citizen, Casio, Tissot, Breitling, Fossil"},
  "football": {"passage": "Nike, Adidas, Puma, Umbro, New Balance, Mitre, Under Armour, Select, Wilson, Diadora"},
  "locks": {"passage": "Yale, Schlage, Kwikset, August, Master Lock, Medeco, Mul-T-Lock, Baldwin, Weiser, Defiant"}
}


In [12]:
# Define the JSON spec from the fake database with text passages
json_spec = JsonSpec(dict_=products, max_value_length=4000)

# Initialize the JSON Toolkit
json_toolkit = JsonToolkit(spec=json_spec)

In [13]:
# Create the JSON Agent
llm = OpenAI(temperature=0)

json_agent_executor = create_json_agent(
    llm=llm,
    toolkit=json_toolkit,
    verbose=True
)

In [14]:
# Ask the question: Get the top 10 brands from the passage
response = json_agent_executor.run(
    "Give me the top 10 brands for shoes, parsing them from the passage as key-value JSON pairs."
)

# Print the response
print(response)



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['shoes', 'watches', 'football', 'locks']
Thought: I should look at the keys that exist in data["shoes"] to see what I have access to
Action: json_spec_list_keys
Action Input: data["shoes"]
Observation: ['passage']
Thought: I should look at the keys that exist in data["shoes"]["passage"] to see what I have access to
Action: json_spec_list_keys
Action Input: data["shoes"]["passage"]
Observation: ValueError('Value at path `data["shoes"]["passage"]` is not a dict, get the value directly.')
Thought: I should get the value at data["shoes"]["passage"]
Action: json_spec_get_value
Action Input: data["shoes"]["passage"]
Observation: Nike, Adidas, Puma, Reebok, Under Armour, Converse, New Balance, Vans, Asics, Fila
Thought: I now know the final answer
Final Answer: Nike, Adidas, Puma, Reebok, Under Armour, Converse, New Balance, Vans, Asics, Fila

> Finished chain.
Nike, Adidas, Puma, Reebok, Unde

In [19]:
# Sample JSON with passage
passage_data = {
  "passage": "Globally, when considering **shoes**, **Nike** and **Adidas** are unequivocally at the pinnacle, consistently holding the top ranks for both quality and widespread use. Following closely, **Puma** and **Under Armour** are generally perceived as high-ranking alternatives. Brands like **Reebok**, **New Balance**, and **Asics** typically occupy the next tier, known for their quality and significant global presence. **Converse**, **Vans**, and **Fila**, while highly popular, often fall into a subsequent category in terms of overall global ranking and perceived premium quality compared to the top contenders.\n\nIn the world of **watches**, **Rolex**, **Omega**, and **Cartier** are universally understood to represent the zenith of luxury and quality, consistently occupying the top-tier rankings. **Tag Heuer** and **Breitling** are generally seen as prestigious, high-ranking alternatives. **Seiko** and **Citizen** are widely recognized for their excellent value and reliability, typically ranking in the mid-to-upper tiers. **Casio** is known for its affordability and mass appeal, usually ranking in the mid-to-lower tiers, while **Tissot** and **Fossil** often occupy similar positions, focusing on style and accessibility.\n\nFor **football** apparel and equipment, **Nike** and **Adidas** again dominate the top rankings globally in terms of both quality and usage. **Puma** and **Umbro** are generally considered strong, high-ranking competitors. **New Balance** and **Under Armour** are also significant players, typically ranking in the mid-to-upper tiers. Brands like **Mitre**, **Select**, **Wilson**, and **Diadora**, while having their own loyal customer bases, often rank in the mid-to-lower tiers in terms of overall global market share and perceived top-tier quality.\n\nRegarding **locks**, **Yale**, **Schlage**, and **Kwikset** are generally recognized as reliable and widely used, often ranking high in terms of accessibility and general security. For higher security needs, **Medeco** and **Mul-T-Lock** are typically perceived as top-tier options, ranking highest in security features. **August**, focusing on smart home integration, represents a different category but is generally well-regarded. **Baldwin** and **Weiser** often rank in the mid-to-upper tiers for their blend of security and style, while **Master Lock** and **Defiant** are usually positioned in the mid-to-lower tiers, often prioritizing affordability and basic security."
}

In [26]:
# Create JSON Spec
json_spec = JsonSpec(dict_=passage_data, max_value_length=4000)

# Initialize Toolkit
json_toolkit = JsonToolkit(spec=json_spec)

In [25]:
# Create Agent
llm = OpenAI(temperature=0)

json_agent_executor = create_json_agent(
    llm=llm,
    toolkit=json_toolkit,
    verbose=True
)

In [23]:
# Ask: Parse passage carefully and list top 10 brands for "shoes"
response = json_agent_executor.run(
    "From the passage, extract the top 5 brands for the 'locks' category based on their described quality and ranking."
)

# Output
print(response)



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['passage']
Thought: I should look at the keys that exist in data["passage"] to see what I have access to
Action: json_spec_list_keys
Action Input: data["passage"]
Observation: ValueError('Value at path `data["passage"]` is not a dict, get the value directly.')
Thought: I should use json_spec_get_value to see what the value is at data["passage"]
Action: json_spec_get_value
Action Input: data["passage"]
Observation: Globally, when considering **shoes**, **Nike** and **Adidas** are unequivocally at the pinnacle, consistently holding the top ranks for both quality and widespread use. Following closely, **Puma** and **Under Armour** are generally perceived as high-ranking alternatives. Brands like **Reebok**, **New Balance**, and **Asics** typically occupy the next tier, known for their quality and significant global presence. **Converse**, **Vans**, and **Fila**, while highly popular, often

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4361 tokens (4105 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}